# Brick Model Fault Detection & Visualization

Run all Brick-mapped rules on AHU7 data, then **zoom in on fault events** to inspect them. This notebook sets up the next tutorial: **working with false positives**.

## Workflow
1. Load Brick TTL, resolve column map, run rules (same as `run_all_rules_brick.py`)
2. Extract fault *events* (contiguous fault regions)
3. Randomly sample events and zoom in with plots
4. Inspect signals during fault windows — ready for false-positive analysis

## 1. Run Brick-driven fault detection

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np

# Paths: run from project root, examples/, or examples/brick_fault_viz/
cwd = Path(".").resolve()
if (cwd / "examples" / "brick_model.ttl").exists():
    EXAMPLES = cwd / "examples"
    ROOT = cwd
elif (cwd / "brick_model.ttl").exists():
    EXAMPLES = cwd
    ROOT = cwd.parent
else:
    EXAMPLES = cwd.parent  # notebook in examples/brick_fault_viz/
    ROOT = EXAMPLES.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from open_fdd.engine.brick_resolver import resolve_from_ttl, get_equipment_types_from_ttl
from open_fdd.engine.runner import RuleRunner, load_rules_from_dir

In [ ]:
def _filter_rules_by_equipment(rules, equipment_types):
    if not equipment_types:
        return rules
    return [r for r in rules if not r.get("equipment_type") or any(et in equipment_types for et in r.get("equipment_type", []))]

def _add_synthetic_columns(df, column_map):
    df = df.copy()
    for brick_key, csv_col in column_map.items():
        if csv_col not in df.columns and ("Setpoint" in brick_key or "setpoint" in brick_key.lower()):
            df[csv_col] = 0.5
    return df

In [ ]:
ttl_path = EXAMPLES / "brick_model.ttl"
rules_dir = EXAMPLES / "my_rules"
csv_path = EXAMPLES / "data_ahu7.csv"

column_map = resolve_from_ttl(ttl_path)
equipment_types = get_equipment_types_from_ttl(ttl_path)
print(f"Column map: {len(column_map)} mappings | Equipment: {equipment_types}")

all_rules = load_rules_from_dir(rules_dir)
rules = _filter_rules_by_equipment(all_rules, equipment_types)
print(f"Rules: {len(rules)} apply to this equipment")

df = pd.read_csv(csv_path)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = _add_synthetic_columns(df, column_map)

runner = RuleRunner(rules=rules)
result = runner.run(df, timestamp_col="timestamp", params={"units": "imperial"}, skip_missing_columns=True, column_map=column_map)

flag_cols = [c for c in result.columns if c.endswith("_flag")]
print(f"\nFlag columns: {flag_cols}")
for col in flag_cols:
    print(f"  {col}: {int(result[col].sum())} fault samples")

## 2. Extract fault events (contiguous regions)

Each fault flag is a boolean series. An *event* is a contiguous run of `True` values.

In [ ]:
def get_fault_events(df, flag_col):
    """Return list of (start_iloc, end_iloc, flag_name) for contiguous fault regions."""
    s = df[flag_col].astype(bool)
    if not s.any():
        return []
    groups = (~s).cumsum()
    fault_groups = groups[s]
    events = []
    for g in fault_groups.unique():
        idx = fault_groups[fault_groups == g].index
        pos = df.index.get_indexer(idx)
        events.append((int(pos.min()), int(pos.max()), flag_col))
    return events

def all_fault_events(df, flag_cols):
    """Collect events from all flag columns."""
    events = []
    for col in flag_cols:
        events.extend(get_fault_events(df, col))
    return sorted(events, key=lambda e: e[0])

In [ ]:
events = all_fault_events(result, flag_cols)
print(f"Total fault events: {len(events)}")
for col in flag_cols:
    n = len([e for e in events if e[2] == col])
    print(f"  {col}: {n} events")

## 3. Zoom in on random fault events

Pick random events and plot the time window around them. Signals + fault shading.

In [ ]:
# Key signals to plot (Brick-mapped column names from TTL)
SIGNAL_COLS = [
    "SAT (°F)", "MAT (°F)", "OAT (°F)", "RAT (°F)",
    "SA Static Press (inH₂O)", "SF Spd Cmd (%)", "OA Damper Cmd (%)",
    "Clg Vlv Cmd (%)", "Prht Vlv Cmd (%)",
]
# Use whatever exists in result
plot_cols = [c for c in SIGNAL_COLS if c in result.columns]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def zoom_on_event(df, event, pad=24, signal_cols=None):
    """Plot signals in a window around a fault event. pad = samples before/after."""
    start_iloc, end_iloc, flag_name = event
    center = (start_iloc + end_iloc) // 2
    lo = max(0, center - pad)
    hi = min(len(df) - 1, center + pad)
    window = df.iloc[lo : hi + 1]
    
    if signal_cols is None:
        signal_cols = [c for c in ["SAT (°F)", "MAT (°F)", "OAT (°F)", "RAT (°F)", "SA Static Press (inH₂O)", "SF Spd Cmd (%)", "OA Damper Cmd (%)"] if c in df.columns]
    
    n_axes = len(signal_cols) + 1  # +1 for fault flag
    fig, axes = plt.subplots(n_axes, 1, figsize=(12, 2 * n_axes), sharex=True)
    if n_axes == 1:
        axes = [axes]
    
    ts = window["timestamp"] if "timestamp" in window.columns else window.index
    
    for ax, col in zip(axes[:-1], signal_cols):
        if col in window.columns:
            ax.plot(ts, window[col], color="#2e86ab", linewidth=1.2)
        ax.set_ylabel(col, fontsize=9)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis="x", labelsize=8)
    
    # Fault flag
    flag_vals = window[flag_name] if flag_name in window.columns else pd.Series(0, index=window.index)
    axes[-1].fill_between(ts, 0, flag_vals, color="#e94f37", alpha=0.6, step="post")
    axes[-1].set_ylabel(flag_name, fontsize=9)
    axes[-1].set_ylim(-0.1, 1.2)
    axes[-1].grid(True, alpha=0.3)
    
    # Shade fault region on all axes (positions within window)
    fault_lo = max(0, start_iloc - lo)
    fault_hi = min(len(window) - 1, end_iloc - lo)
    if fault_lo <= fault_hi:
        for ax in axes[:-1]:
            ax.axvspan(ts.iloc[fault_lo], ts.iloc[fault_hi], alpha=0.15, color="#e94f37")
    
    fig.suptitle(f"Zoom: {flag_name} @ iloc {start_iloc}-{end_iloc}", fontsize=11)
    plt.tight_layout()
    return fig

In [ ]:
import random
random.seed(42)
n_sample = 3  # number of random events to plot
sampled = random.sample(events, min(n_sample, len(events)))

for event in sampled:
    zoom_on_event(result, event, pad=48, signal_cols=plot_cols)
    plt.show()

## 4. Event summary table

Quick lookup: when did each fault type occur?

In [ ]:
event_rows = []
for start_iloc, end_iloc, flag_name in events[:50]:  # first 50
    t0 = result.iloc[start_iloc]["timestamp"] if "timestamp" in result.columns else start_iloc
    t1 = result.iloc[end_iloc]["timestamp"] if "timestamp" in result.columns else end_iloc
    duration = end_iloc - start_iloc + 1
    event_rows.append({"flag": flag_name, "start": t0, "end": t1, "duration_samples": duration})

pd.DataFrame(event_rows)

## Next: False positives

Many fault flags can be **false positives** — the rule fired but the condition was acceptable (e.g. startup, setpoint change, sensor noise). The next tutorial covers:
- Filtering by occupancy / schedule
- Rolling-window confirmation
- Manual review workflows
- Tuning thresholds to reduce false positives